In [0]:
#import required libraries

from pyspark.sql.types import StructType,StructField,IntegerType,StringType

In [0]:
#defining a schema

schema=StructType([
                           StructField('File',StringType(),True),
                           StructField('Shop',StringType(),True),
                           StructField('Sale_count',IntegerType(),True)
                          ])

In [0]:
#commands to remove and create a directory in the dbfs

dbutils.fs.mkdirs("/FileStore/tables/stream_checkpoint/")
dbutils.fs.mkdirs("/FileStore/tables/stream_read/")
dbutils.fs.mkdirs("/FileStore/tables/stream_write/")
# dbutils.fs.rm("/FileStore/tables/stream_checkpoint/",True)
# dbutils.fs.rm("/FileStore/tables/stream_read/",True)
# dbutils.fs.rm("/FileStore/tables/stream_write/",True)

Out[14]: True

In [0]:
#to read the data in realtime from the stream_read location
#the data is defined in schema

df=spark.readStream.format("csv").schema(schema).option("header",True).option("sep",",").load("/FileStore/tables/stream_read/")

#>>to process the data in realtime from the read stream and display the sum
#(Transformation)
# df1=df.groupBy("Shop").sum("Sale_count")
# display(df1)

In [0]:
#writing the data as parquet file with checkpoint in the checkpoint folder

df2=df.writeStream.format("parquet").outputMode("append").option("path","/FileStore/tables/stream_write/").option("checkpointLocation","/FileStore/tables/stream_checkpoint/").start().awaitTermination()

In [0]:
#verifying the written file data

df3=spark.read.format("parquet").load("/FileStore/tables/stream_write/*.parquet")

In [0]:
display(df3)

File Shop Sale_count File1 Amazon 100 File1 Flipkart 50 File1 Myntra 20 File1 Snapdeal 10 File1 Alibaba null File5 Amazon 100 File5 Flipkart 50 File5 Myntra 20 File5 Snapdeal 10 File5 Alibaba null File3 Amazon 100 File3 Flipkart 50 File3 Myntra 20 File3 Snapdeal 10 File3 Alibaba null File2 Amazon 100 File2 Flipkart 50 File2 Myntra 20 File2 Snapdeal 10 File2 Alibaba null File4 Amazon 100 File4 Flipkart 50 File4 Myntra 20 File4 Snapdeal 10 File4 Alibaba null